In [1]:
# Basic Statistics tasks

#     Which store has maximum sales?
#     Which store has maximum standard deviation i.e., the sales vary a lot. Also, find out the coefficient of mean to standard deviation
#     Which store/s has good quarterly growth rate in Q3’2012?
#     Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together
#     Provide a monthly and semester view of sales in units and give insights?

# Statistical Model

# For Store 1 – Build  prediction models to forecast demand
#     Linear Regression – Utilize variables like date and restructure dates as 1 for 5 Feb 2010 (starting from the earliest date in order). Hypothesize if CPI, unemployment, and fuel price have any impact on sales.
#     Change dates into days by creating new variable.
# Select the model which gives best accuracy.

In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from patsy import dmatrices
import sklearn
import seaborn as sns

In [7]:
sells = pd.read_csv("../../../ai_algos/data/Walmart_Store_sales.csv")

In [10]:
sells.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [ ]:
# Dataset Description

# This is the historical data that covers sales from 2010-02-05 to 2012-11-01, in the file Walmart_Store_sales. Within this file you will find the following fields:

#     Store - the store number
#     Date - the week of sales
#     Weekly_Sales -  sales for the given store
#     Holiday_Flag - whether the week is a special holiday week 1 – Holiday week 0 – Non-holiday week
#     Temperature - Temperature on the day of sale
#     Fuel_Price - Cost of fuel in the region
#     CPI – Prevailing consumer price index
#     Unemployment - Prevailing unemployment rate

# Holiday Events

# Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
# Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
# Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
# Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [9]:
sells.isnull().values.any()

False